<a href="https://colab.research.google.com/github/yuseongil034/yuseongil/blob/main/0718_yolov12.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Google Colab용 YouTube YOLO 12 영상 처리기 (최적화)
# 이 코드를 하나의 셀에서 실행하세요

# 1. 필요한 패키지 설치
print("📦 패키지 설치 중...")
!pip install ultralytics yt-dlp opencv-python -q

# 2. 라이브러리 임포트
import cv2
import yt_dlp
import os
import time
from pathlib import Path
from typing import Optional, Dict, Any

class YOLOVideoProcessor:
    """YouTube 영상 처리기 with YOLOv12 객체 탐지"""

    def __init__(self, model_path: str = "yolo12n.pt"):
        """처리기 초기화"""
        self.model_path = model_path
        self.model = None

    def load_model(self) -> bool:
        """YOLOv12 모델 로드"""
        try:
            from ultralytics import YOLO
            print(f"🤖 YOLOv12 모델 로드 중: {self.model_path}")
            self.model = YOLO(self.model_path)
            print(f"✅ 모델 로드 완료!")
            return True
        except Exception as e:
            print(f"❌ 모델 로드 실패: {e}")
            return False

    def get_video_info(self, youtube_url: str) -> Optional[Dict[str, Any]]:
        """YouTube 영상 정보 가져오기"""
        try:
            with yt_dlp.YoutubeDL({'quiet': True}) as ydl:
                info = ydl.extract_info(youtube_url, download=False)
                return {
                    'title': info.get('title', '알 수 없음'),
                    'duration': info.get('duration', 0)
                }
        except Exception as e:
            print(f"❌ 영상 정보 가져오기 실패: {e}")
            return None

    def format_time(self, seconds: float) -> str:
        """초를 MM:SS 형식으로 변환"""
        minutes = int(seconds // 60)
        seconds = int(seconds % 60)
        return f"{minutes:02d}:{seconds:02d}"

    def process_video(self, youtube_url: str, output_filename: str = "yolo_output.mp4",
                     start_time: float = 0, duration: int = 10, skip_frames: int = 2) -> bool:
        """
        YouTube 영상을 YOLO 12 탐지로 처리

        Args:
            youtube_url: YouTube 영상 URL
            output_filename: 출력 파일명
            start_time: 시작 시간(초)
            duration: 처리할 길이(초)
            skip_frames: 건너뛸 프레임 수
        """
        if not self.model and not self.load_model():
            return False

        # 다운로드 설정 (720p 이하로 제한)
        ydl_opts = {
            'format': 'mp4/best[height<=720]',
            'outtmpl': '/tmp/temp_video.%(ext)s',
            'quiet': True,
        }

        temp_video_path = None
        cap = None
        out = None

        try:
            # 영상 다운로드
            print("⬇️ YouTube 영상 다운로드 중...")
            with yt_dlp.YoutubeDL(ydl_opts) as ydl:
                info = ydl.extract_info(youtube_url, download=True)
                temp_video_path = ydl.prepare_filename(info)
                print(f"🎥 제목: {info['title']}")

            # 비디오 캡처 초기화
            cap = cv2.VideoCapture(temp_video_path)
            if not cap.isOpened():
                raise ValueError("영상 파일을 열 수 없습니다")

            # 영상 속성
            fps = cap.get(cv2.CAP_PROP_FPS)
            frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
            frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
            total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
            video_duration = total_frames / fps

            # 시간 범위 설정
            end_time = min(start_time + duration, video_duration)
            start_frame = int(start_time * fps)
            end_frame = int(end_time * fps)

            print(f"📹 영상 정보: {frame_width}x{frame_height}, {fps:.1f} FPS")
            print(f"🎯 처리 구간: {self.format_time(start_time)} ~ {self.format_time(end_time)}")

            # 시작 프레임으로 이동
            cap.set(cv2.CAP_PROP_POS_FRAMES, start_frame)

            # 비디오 라이터 초기화
            fourcc = cv2.VideoWriter_fourcc(*'mp4v')
            output_fps = fps / skip_frames
            out = cv2.VideoWriter(output_filename, fourcc, output_fps, (frame_width, frame_height))

            # 프레임 처리
            frame_num = start_frame
            processed_frames = 0
            total_detections = 0
            detected_classes = set()
            start_processing_time = time.time()

            print("🔄 YOLO 12 추론 시작...")

            while True:
                ret, frame = cap.read()
                if not ret or frame_num >= end_frame:
                    break

                # 프레임 건너뛰기
                if (frame_num - start_frame) % skip_frames == 0:
                    try:
                        # YOLO 12 추론 (최적화된 설정)
                        results = self.model.predict(
                            frame,
                            imgsz=640,
                            verbose=False,
                            show=False,
                            save=False,
                            conf=0.25,  # 신뢰도 임계값
                            iou=0.45    # NMS IOU 임계값
                        )[0]

                        # 결과 시각화
                        annotated_frame = results.plot()
                        out.write(annotated_frame)
                        processed_frames += 1

                        # 통계 업데이트
                        if results.boxes is not None:
                            detections = len(results.boxes)
                            total_detections += detections

                            # 탐지된 클래스 추적
                            for box in results.boxes:
                                class_id = int(box.cls[0])
                                class_name = self.model.names[class_id]
                                detected_classes.add(class_name)

                        # 진행률 표시 (매 30프레임마다)
                        if processed_frames % 30 == 0:
                            progress = ((frame_num - start_frame) / (end_frame - start_frame)) * 100
                            print(f"⏳ 진행률: {progress:.1f}% | 처리 프레임: {processed_frames}")

                    except Exception as e:
                        print(f"⚠️ 프레임 {frame_num} 처리 중 오류: {e}")
                        continue

                frame_num += 1

            # 최종 통계
            total_time = time.time() - start_processing_time

            print(f"\n✅ YOLO 12 영상 추론 완료!")
            print(f"📊 처리 시간: {total_time:.2f}초")
            print(f"📊 처리된 프레임: {processed_frames}")
            print(f"📊 총 탐지 수: {total_detections}")
            print(f"📊 탐지된 클래스: {', '.join(sorted(detected_classes))}")
            print(f"💾 출력 파일: {output_filename}")

            # 파일 크기 확인
            if os.path.exists(output_filename):
                file_size = os.path.getsize(output_filename) / (1024*1024)
                print(f"📁 파일 크기: {file_size:.2f} MB")

            return True

        except Exception as e:
            print(f"❌ 오류 발생: {e}")
            return False

        finally:
            # 리소스 정리
            if cap:
                cap.release()
            if out:
                out.release()
            if temp_video_path and os.path.exists(temp_video_path):
                try:
                    os.remove(temp_video_path)
                    print("🗑️ 임시 파일 정리 완료")
                except:
                    pass

# 빠른 실행을 위한 함수
def quick_process(youtube_url: str, start_time: float = 0, duration: int = 10):
    """빠른 처리를 위한 간단한 함수"""
    print("🚀 빠른 YOLO 12 영상 처리 시작!")
    processor = YOLOVideoProcessor("yolo12n.pt")
    return processor.process_video(
        youtube_url=youtube_url,
        start_time=start_time,
        duration=duration,
        skip_frames=2
    )

# Google Colab에서 다운로드 기능
def download_result(filename: str = "yolo_output.mp4"):
    """결과 파일 다운로드"""
    try:
        from google.colab import files
        if os.path.exists(filename):
            files.download(filename)
            print(f"📥 {filename} 다운로드 시작!")
        else:
            print(f"❌ {filename} 파일을 찾을 수 없습니다.")
    except ImportError:
        print("❌ Google Colab 환경이 아닙니다.")

# 사용 예시
def example_usage():
    """사용 예시"""
    print("💡 사용 예시:")
    print("quick_process('https://youtube.com/watch?v=...', start_time=30, duration=10)")
    print("\n또는:")
    print("processor = YOLOVideoProcessor('yolo12n.pt')")
    print("processor.process_video('https://youtube.com/watch?v=...', start_time=0, duration=15)")

# 즉시 실행
print("🎉 YOLO 12 영상 처리기 준비 완료!")
print("=" * 50)
example_usage()
print("=" * 50)
print("📥 결과 다운로드: download_result('yolo_output.mp4')")

📦 패키지 설치 중...
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 174.3/174.3 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 39.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 85.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 66.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 36.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 45.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
# 필요한 패키지 설치
# !pip install ultralytics yt-dlp opencv-python

import cv2
import yt_dlp
import tempfile
import os
import time
import logging
from pathlib import Path
from typing import Optional, Tuple, List, Dict, Any
import sys

# 로깅 설정
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

# Jupyter 환경 감지
try:
    from IPython.display import clear_output
    JUPYTER_ENV = True
except ImportError:
    JUPYTER_ENV = False

class YOLOVideoProcessor:
    """간소화된 YouTube 영상 처리기 with YOLOv12 객체 탐지"""

    def __init__(self, model_path: str = "yolo12n.pt"):
        self.model_path = model_path
        self.model = None

    def load_model(self) -> bool:
        """YOLOv12 모델 로드"""
        try:
            from ultralytics import YOLO
            self.model = YOLO(self.model_path)
            logger.info(f"🤖 YOLOv12 모델 로드 완료: {self.model_path}")
            return True
        except Exception as e:
            logger.error(f"❌ YOLOv12 모델 로드 실패: {e}")
            return False

    def format_time(self, seconds: float) -> str:
        """초를 MM:SS 형식으로 변환"""
        minutes = int(seconds // 60)
        seconds = int(seconds % 60)
        return f"{minutes:02d}:{seconds:02d}"

    def process_video(self, youtube_url: str, output_filename: str = "yolo12_output.mp4",
                     start_time: float = 0, end_time: Optional[float] = None,
                     max_duration: int = 10, skip_frames: int = 5) -> bool:
        """YouTube 영상을 YOLOv12 탐지로 처리"""

        if not self.model and not self.load_model():
            return False

        # 출력 디렉토리 생성
        output_path = Path(output_filename)
        output_path.parent.mkdir(parents=True, exist_ok=True)

        # 다운로드 설정
        ydl_opts = {
            'format': 'mp4/best[height<=1080]',
            'outtmpl': os.path.join(tempfile.gettempdir(), 'temp_video.%(ext)s'),
            'quiet': True,
        }

        temp_video_path = None
        cap = None
        out = None

        try:
            # 영상 다운로드
            with yt_dlp.YoutubeDL(ydl_opts) as ydl:
                logger.info("⬇️ YouTube 영상 다운로드 중...")
                info = ydl.extract_info(youtube_url, download=True)
                temp_video_path = ydl.prepare_filename(info)
                logger.info(f"🎥 제목: {info['title']}")
                logger.info(f"📊 길이: {info.get('duration', '알 수 없음')}초")

            # 비디오 캡처 초기화
            cap = cv2.VideoCapture(temp_video_path)
            if not cap.isOpened():
                raise ValueError("영상 파일을 열 수 없습니다")

            # 영상 속성 가져오기
            fps = cap.get(cv2.CAP_PROP_FPS)
            frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
            frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
            total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
            video_duration = total_frames / fps

            # 시간 범위 설정
            if end_time is None:
                end_time = min(start_time + max_duration, video_duration)

            start_time = max(0, start_time)
            end_time = min(end_time, video_duration)

            if start_time >= end_time:
                logger.warning(f"잘못된 시간 범위, 마지막 10초 사용")
                end_time = video_duration
                start_time = max(0, video_duration - 10)

            # 프레임 번호로 변환
            start_frame = int(start_time * fps)
            end_frame = int(end_time * fps)

            logger.info(f"📹 영상 정보: {frame_width}x{frame_height}, {fps:.2f} FPS")
            logger.info(f"🎯 처리 구간: {self.format_time(start_time)} ~ {self.format_time(end_time)}")

            # 시작 프레임으로 이동
            if start_frame > 0:
                cap.set(cv2.CAP_PROP_POS_FRAMES, start_frame)

            # 비디오 라이터 초기화
            fourcc = cv2.VideoWriter_fourcc(*'mp4v')
            output_fps = fps / skip_frames
            out = cv2.VideoWriter(output_filename, fourcc, output_fps, (frame_width, frame_height))

            if not out.isOpened():
                raise ValueError("출력 영상 파일을 생성할 수 없습니다")

            # 프레임 처리
            frame_num = start_frame
            processed_frames = 0
            start_processing_time = time.time()

            logger.info("🔄 YOLOv12 추론 시작...")

            while True:
                ret, frame = cap.read()
                if not ret or frame_num >= end_frame:
                    break

                # 프레임 건너뛰기
                if (frame_num - start_frame) % skip_frames == 0:
                    try:
                        # YOLOv12 추론
                        results = self.model.predict(
                            frame,
                            verbose=False,
                            show=False,
                            save=False
                        )[0]

                        # 결과 시각화
                        annotated_frame = results.plot()
                        out.write(annotated_frame)
                        processed_frames += 1

                        # 진행률 표시
                        if processed_frames % 30 == 0:
                            progress = ((frame_num - start_frame) / (end_frame - start_frame)) * 100
                            current_time = frame_num / fps
                            logger.info(f"⏳ 진행률: {progress:.1f}% | 시간: {self.format_time(current_time)}")

                    except Exception as e:
                        logger.warning(f"⚠️ 프레임 {frame_num} 처리 중 오류: {e}")
                        continue

                frame_num += 1

            # 완료 정보
            total_time = time.time() - start_processing_time
            logger.info(f"✅ YOLOv12 영상 추론 완료!")
            logger.info(f"📊 처리 시간: {total_time:.2f}초")
            logger.info(f"📊 처리된 프레임: {processed_frames}")
            logger.info(f"📊 평균 처리 FPS: {processed_frames/total_time:.1f}")
            logger.info(f"💾 출력 파일: {output_filename}")

            # 파일 크기 확인
            if os.path.exists(output_filename):
                file_size = os.path.getsize(output_filename) / (1024*1024)
                logger.info(f"📁 파일 크기: {file_size:.2f} MB")

            return True

        except Exception as e:
            logger.error(f"❌ 오류 발생: {e}")
            return False

        finally:
            # 리소스 정리
            if cap:
                cap.release()
            if out:
                out.release()
            if temp_video_path and os.path.exists(temp_video_path):
                try:
                    os.remove(temp_video_path)
                    logger.info("🗑️ 임시 파일 정리 완료")
                except:
                    pass


def get_available_models() -> Dict[str, str]:
    """사용 가능한 YOLOv12 모델 목록"""
    return {
        'yolo12n.pt': 'YOLOv12 Nano - 가장 빠르고 가벼운 모델',
        'yolo12s.pt': 'YOLOv12 Small - 속도와 정확도의 균형',
        'yolo12m.pt': 'YOLOv12 Medium - 중간 크기 모델',
        'yolo12l.pt': 'YOLOv12 Large - 높은 정확도',
        'yolo12x.pt': 'YOLOv12 Extra Large - 최고 정확도'
    }


def interactive_setup() -> Tuple[str, str, float, Optional[float]]:
    """간소화된 대화형 설정 인터페이스"""
    print("\n🎥 YouTube YOLOv12 영상 처리기")
    print("=" * 50)

    # YouTube URL 입력
    youtube_url = input("📋 YouTube 영상 URL을 입력하세요: ").strip()
    if not youtube_url:
        raise ValueError("URL이 필요합니다")

    # 모델 선택
    models = get_available_models()
    print("\n🤖 사용 가능한 YOLOv12 모델:")
    for i, (model_name, description) in enumerate(models.items(), 1):
        print(f"{i}. {model_name} - {description}")

    while True:
        try:
            choice = input(f"\n모델을 선택하세요 (1-{len(models)}, 기본값: 1): ").strip()
            if not choice:
                choice = "1"
            choice = int(choice)
            if 1 <= choice <= len(models):
                model_path = list(models.keys())[choice-1]
                break
            else:
                print(f"❌ 1부터 {len(models)} 사이의 숫자를 입력하세요")
        except ValueError:
            print("❌ 올바른 숫자를 입력하세요")

    # 영상 정보 가져오기
    processor = YOLOVideoProcessor()
    ydl_opts = {'quiet': True, 'no_warnings': True}

    try:
        with yt_dlp.YoutubeDL(ydl_opts) as ydl:
            info = ydl.extract_info(youtube_url, download=False)
            duration = info.get('duration', 0)
            print(f"\n📹 제목: {info['title']}")
            print(f"⏱ 길이: {duration}초 ({processor.format_time(duration)})")
            print(f"👤 업로더: {info['uploader']}")
    except Exception as e:
        logger.error(f"❌ 영상 정보 가져오기 실패: {e}")
        duration = 300  # 기본값 5분

    # 시간 범위 선택
    print(f"\n⏰ 처리할 구간을 선택하세요 (0~{duration}초)")
    start_time = float(input(f"🎬 시작 시간 (초, 기본값: 0): ") or "0")
    end_input = input(f"🏁 종료 시간 (초, 기본값: 자동): ").strip()
    end_time = float(end_input) if end_input else None

    return youtube_url, model_path, start_time, end_time


def main():
    """메인 함수"""
    # Jupyter 환경 체크
    if JUPYTER_ENV or any('-f' in arg for arg in sys.argv):
        sys.argv = [sys.argv[0]]

    try:
        # 대화형 모드
        youtube_url, model_path, start_time, end_time = interactive_setup()
        output_filename = input("💾 출력 파일명 (기본값: yolo12_output.mp4): ") or "yolo12_output.mp4"
        skip_frames = int(input("⏭ 건너뛸 프레임 수 (기본값: 5): ") or "5")

        # 출력 파일 확장자 확인
        if not output_filename.endswith('.mp4'):
            output_filename += '.mp4'

        # 영상 처리
        processor = YOLOVideoProcessor(model_path)
        success = processor.process_video(
            youtube_url=youtube_url,
            output_filename=output_filename,
            start_time=start_time,
            end_time=end_time,
            max_duration=10,
            skip_frames=skip_frames
        )

        if success:
            print("\n🎉 처리가 성공적으로 완료되었습니다!")
        else:
            print("\n❌ 처리 실패!")

    except KeyboardInterrupt:
        print("\n🚫 사용자에 의해 중단되었습니다")
    except Exception as e:
        logger.error(f"❌ 예상치 못한 오류: {e}")


# 빠른 실행을 위한 함수
def quick_process(youtube_url: str, start_time: float = 0, duration: int = 10):
    """빠른 처리를 위한 간단한 함수"""
    processor = YOLOVideoProcessor("yolo12n.pt")
    return processor.process_video(
        youtube_url=youtube_url,
        start_time=start_time,
        max_duration=duration,
        skip_frames=3
    )


# 고급 처리를 위한 함수
def advanced_process(youtube_url: str, model_size: str = "s", start_time: float = 0,
                    duration: int = 10, output_filename: str = "yolo12_advanced.mp4"):
    """고급 처리를 위한 함수"""
    model_map = {
        "n": "yolo12n.pt",
        "s": "yolo12s.pt",
        "m": "yolo12m.pt",
        "l": "yolo12l.pt",
        "x": "yolo12x.pt"
    }

    model_path = model_map.get(model_size, "yolo12s.pt")
    processor = YOLOVideoProcessor(model_path)

    return processor.process_video(
        youtube_url=youtube_url,
        output_filename=output_filename,
        start_time=start_time,
        max_duration=duration,
        skip_frames=2
    )


# 배치 처리를 위한 함수
def batch_process(video_urls: List[str], output_dir: str = "yolo12_batch",
                  model_size: str = "n", duration: int = 10):
    """여러 영상을 배치 처리하는 함수"""
    os.makedirs(output_dir, exist_ok=True)

    model_map = {
        "n": "yolo12n.pt",
        "s": "yolo12s.pt",
        "m": "yolo12m.pt",
        "l": "yolo12l.pt",
        "x": "yolo12x.pt"
    }

    model_path = model_map.get(model_size, "yolo12n.pt")
    processor = YOLOVideoProcessor(model_path)

    results = []
    for i, url in enumerate(video_urls):
        try:
            logger.info(f"📹 처리 중: {i+1}/{len(video_urls)} - {url}")
            output_filename = os.path.join(output_dir, f"video_{i+1:03d}_yolo12.mp4")

            success = processor.process_video(
                youtube_url=url,
                output_filename=output_filename,
                start_time=0,
                max_duration=duration,
                skip_frames=5
            )

            results.append({'url': url, 'output_file': output_filename, 'success': success})

        except Exception as e:
            logger.error(f"❌ 영상 {i+1} 처리 실패: {e}")
            results.append({'url': url, 'output_file': None, 'success': False})

    successful = sum(1 for r in results if r['success'])
    logger.info(f"🎉 배치 처리 완료: {successful}/{len(video_urls)} 성공")
    return results


# Jupyter/Colab에서 직접 실행하기 위한 함수
def run_in_jupyter():
    """Jupyter/Colab에서 직접 실행"""
    try:
        youtube_url, model_path, start_time, end_time = interactive_setup()
        output_filename = input("💾 출력 파일명 (기본값: yolo12_output.mp4): ") or "yolo12_output.mp4"
        skip_frames = int(input("⏭ 건너뛸 프레임 수 (기본값: 5): ") or "5")

        if not output_filename.endswith('.mp4'):
            output_filename += '.mp4'

        processor = YOLOVideoProcessor(model_path)
        success = processor.process_video(
            youtube_url=youtube_url,
            output_filename=output_filename,
            start_time=start_time,
            end_time=end_time,
            max_duration=10,
            skip_frames=skip_frames
        )

        if success:
            print("\n🎉 처리가 성공적으로 완료되었습니다!")
        else:
            print("\n❌ 처리 실패!")

    except Exception as e:
        logger.error(f"❌ 오류 발생: {e}")


if __name__ == "__main__":
    main()


🎥 YouTube YOLOv12 영상 처리기
📋 YouTube 영상 URL을 입력하세요: https://youtu.be/JHva65QGlzE?si=fo4yuZFmiemoB57g

🤖 사용 가능한 YOLOv12 모델:
1. yolo12n.pt - YOLOv12 Nano - 가장 빠르고 가벼운 모델
2. yolo12s.pt - YOLOv12 Small - 속도와 정확도의 균형
3. yolo12m.pt - YOLOv12 Medium - 중간 크기 모델
4. yolo12l.pt - YOLOv12 Large - 높은 정확도
5. yolo12x.pt - YOLOv12 Extra Large - 최고 정확도

모델을 선택하세요 (1-5, 기본값: 1): 1

📹 제목: 서울의 홍대 홍익대 입구 근처의 최근 모습을 감상하세요 Travel destinations in Korea 韓国の目的地 4K
⏱ 길이: 177초 (02:57)
👤 업로더: Seoul_driver

⏰ 처리할 구간을 선택하세요 (0~177초)
🎬 시작 시간 (초, 기본값: 0): 0
🏁 종료 시간 (초, 기본값: 자동): 30
💾 출력 파일명 (기본값: yolo12_output.mp4): yolo12_qlry
⏭ 건너뛸 프레임 수 (기본값: 5): 5
Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


100%|██████████| 5.34M/5.34M [00:00<00:00, 80.8MB/s]



🎉 처리가 성공적으로 완료되었습니다!
